In [ ]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import quote

#------------------------------------------------------------------------------------------------------#
                            # This part is used to fetch the cnsx symbol #

def Fetch_Company_Name(eq):
  url='https://www.google.com/search?q=site:Bloomberg.com '+eq.split()[0]+':IN'
  res=requests.get(url)
  name=None
  if res.status_code==200:
    soup=BeautifulSoup(res.content, 'html.parser')
    links=[m.text for m in soup.find_all('a')]
    for j in links:
      if j.endswith(f'{eq.split()[0]}:IN') and 'Ltd' in j:
        name=j.split('Ltd')[0]+"Ltd"
      if not name and j.endswith(f'{eq.split()[0]}:IN') and 'Stock Price Quote' in j:
        name=j.split(' Stock Price Quote')[0]
      if name and  ': ' in name:
        name=name.split(': ')[1]
  return name

def Fetch_original_name(name):
  url='https://www.google.com/search?q=site%3Awww.google.com%2Ffinance+'+f"{quote(name)}"
  res=requests.get(url)
  if res.status_code==200:
    soup=BeautifulSoup(res.content, 'html.parser')
    links=[m.text for m in soup.find_all('div', {'class':'BNeawe UPmit AP7Wnd lRVwie'})]
    for i in links:
      if i.endswith(':NSE'):
        return(i.split()[-1].split(':')[0])

def Fetch_CNSX(eq):
  name=Fetch_Company_Name(eq)
  url=f'http://www.google.com/finance?&q={name}'
  headers={'User-Agent': 'Mozilla/5.0'}
  resp=requests.get(url,headers=headers)
  title=BeautifulSoup(resp.content, 'html.parser')
  try:
    title=title.find('div',{'class':'PdOqHc'}).contents[-1].split(' • NSE')[0]
  except:
    title=Fetch_original_name(name)
  return title+'.NS'

def Fetch_CNSX_List(x):
  found=[]
  c=0
  try:
    for i in x:
      v=Fetch_CNSX(i)
      c+=1
      found.append((i,v))
      print(f"{i} ---> {v}")
    print("List of Symbols")
    print(found)
  except:
    print("Here is the list till which the names are found")
    print(found)
    print("here is the Remaining list")
    print(x[c:])
    print("PLEASE DISCONNECT AND RE-CONNECT THE COLLAB NOTEBOOK AND PROCESS THE REST.")

#--------------------------------------------------------------------------------------------------------------------#
                                  # Thsi part is used to download the data #

def get_date(d):
    date_object=datetime.strptime(d, '%Y-%m-%d')
    while date_object.strftime('%A')=='Saturday' or date_object.strftime('%A')=='Sunday':
        date_object-=timedelta(days=1)
    return date_object.strftime('%Y-%m-%d')

def stocks_for_each_year(stock_name,f,t):
    data_list=[]
    for i in range(f,t+1):
        date=f'{i}-12-31'
        date=get_date(date)
        date_object = datetime.strptime(date, '%Y-%m-%d')+ timedelta(days=1)
        new_date= date_object.strftime('%Y-%m-%d')
        tickerData = yf.Ticker(stock_name)
        data = tickerData.history(start=date, end=new_date)
        data=pd.DataFrame(data)
        data_list.append(data)
    data_list=pd.concat(data_list)
    return data_list

def stocks_freq(stock_name,f,t,freq):
    data_list=[]
    for i in range(f,t+1):
        start_date=f'{i}-01-01'
        end_date=f'{i}-12-31'
        tickerData = yf.Ticker(stock_name)
        data = tickerData.history(start=start_date, end=end_date, interval=freq)
        data=pd.DataFrame(data)
        data_list.append(data)
    data_list=pd.concat(data_list)
    return data_list

def process_stock(stock,name, f, t, freq):
    if freq=='1Y':
        x = stocks_for_each_year(stock, f,t)
    else:
        x=stocks_freq(stock,f,t,freq)
    x.columns = pd.MultiIndex.from_product([[name], x.columns])
    print(name)
    return x

def display_dataframe(names_and_symbols, f, t, freq):
    names=[]
    symbols=[]
    freq_list, f, t=[freq for i in range (len(names_and_symbols))],[f for i in range (len(names_and_symbols))],[t for i in range (len(names_and_symbols))]
    for i in names_and_symbols:
      names.append(i[0])
      symbols.append(i[1])
    print("Processed:")
    with ThreadPoolExecutor(max_workers=15) as executor:
        results = list(executor.map(process_stock, symbols, names, f, t, freq_list))
    final_dataframe = pd.concat(results, axis=1, join='outer')
    final_dataframe.index = final_dataframe.index.tz_convert(None)
    final_dataframe.to_excel('All_IB_Equity.xlsx')
    print("A excel file named 'All_IB_Equity.xlsx'has been saved to local pc in the same directory.")
    return final_dataframe


In [ ]:
names = ['UPLL IB Equity', 'UTCEM IB Equity', 'VEDL IB Equity', 'VOLT IB Equity', 'WPRO IB Equity', 'Z IB Equity']

Fetch_CNSX_List(names)

NameError: name 'Fetch_CNSX_List' is not defined

In [ ]:
names_and_symbols=[('ACC IB Equity', 'ACC.NS'), ('ACEM IB Equity', 'AMBUJACEM.NS'), ('ADANI IB Equity', 'ADANIPOWER.NS'), ('ADSEZ IB Equity', 'ADANIPORTS.NS'), ('AL IB Equity', 'ASHOKLEY.NS'), ('APHS IB Equity', 'APOLLOHOSP.NS'), ('APNT IB Equity', 'ASIANPAINT.NS'), ('ARBP IB Equity', 'AUROPHARMA.NS'), ('AUBANK IB Equity', 'AUBANK.NS'), ('AXSB IB Equity', 'AXISBANK.NS'), ('BAF IB Equity', 'BAJFINANCE.NS'), ('BANDHAN IB Equity', 'BANDHANBNK.NS'), ('BHARTI IB Equity', 'BHARTIARTL.NS'), ('BHE IB Equity', 'BEL.NS'), ('BHFC IB Equity', 'BHARATFORG.NS'), ('BJAUT IB Equity', 'BAJAJ-AUTO.NS'), ('BJFIN IB Equity', 'BAJAJFINSV.NS'), ('BOB IB Equity', 'BANKBARODA.NS'), ('BPCL IB Equity', 'BPCL.NS'), ('BRIT IB Equity', 'BRITANNIA.NS'), ('CIPLA IB Equity', 'CIPLA.NS'), ('CLGT IB Equity', 'COLPAL.NS'), ('COAL IB Equity', 'COALINDIA.NS'), ('DABUR IB Equity', 'DABUR.NS'), ('DIVI IB Equity', 'DIVISLAB.NS'), ('DLFU IB Equity', 'DLF.NS'), ('DMART IB Equity', 'DMART.NS'), ('DRRD IB Equity', 'DRREDDY.NS'), ('EIM IB Equity', 'EICHERMOT.NS'), ('FB IB Equity', 'FEDERALBNK.NS'), ('GAIL IB Equity', 'GAIL.NS'), ('GCPL IB Equity', 'GODREJCP.NS'), ('GPL IB Equity', 'GODREJPROP.NS'), ('GRASIM IB Equity', 'GRASIM.NS'), ('HAVL IB Equity', 'HAVELLS.NS'), ('HCLT IB Equity', 'HCLTECH.NS'), ('HDFCB IB Equity', 'HDFCBANK.NS'), ('HMCL IB Equity', 'HEROMOTOCO.NS'), ('HNDL IB Equity', 'HINDALCO.NS'), ('HUVR IB Equity', 'HINDUNILVR.NS'), ('ICICIBC IB Equity', 'ICICIBANK.NS'), ('ICICIGI IB Equity', 'ICICIGI.NS'), ('IIB IB Equity', 'INDUSINDBK.NS'), ('INDIGO IB Equity', 'INDIGO.NS'), ('INDUSTOW IB Equity', 'INDUSTOWER.NS'), ('INFO IB Equity', 'INFY.NS'), ('INFOE IB Equity', 'NAUKRI.NS'), ('IOCL IB Equity', 'IOC.NS'), ('IPRU IB Equity', 'ICICIPRULI.NS'), ('IRCTC IB Equity', 'IRCTC.NS'), ('ITC IB Equity', 'ITC.NS'), ('JSTL IB Equity', 'JSWSTEEL.NS'), ('JUBI IB Equity', 'JUBLFOOD.NS'), ('KMB IB Equity', 'KOTAKBANK.NS'), ('LPC IB Equity', 'LUPIN.NS'), ('LT IB Equity', 'LT.NS'), ('LTIM IB Equity', 'LTIM.NS'), ('MAXF IB Equity', 'MFSL.NS'), ('MM IB Equity', 'M&M.NS'), ('MPHL IB Equity', 'MPHASIS.NS'), ('MRCO IB Equity', 'MARICO.NS'), ('MSIL IB Equity', 'MARUTI.NS'), ('NEST IB Equity', 'NESTLEIND.NS'), ('NTPC IB Equity', 'NTPC.NS'), ('ONGC IB Equity', 'ONGC.NS'), ('PAG IB Equity', 'PAGEIND.NS'), ('PI IB Equity', 'PIIND.NS'), ('PIDI IB Equity', 'PIDILITIND.NS'), ('PWGR IB Equity', 'POWERGRID.NS'), ('RIL IB Equity', 'RELIANCE.NS'), ('SBICARD IB Equity', 'SBICARD.NS'), ('SBILIFE IB Equity', 'SBILIFE.NS'), ('SBIN IB Equity', 'SBIN.NS'), ('SIEM IB Equity', 'SIEMENS.NS'), ('SRCM IB Equity', 'SHREECEM.NS'), ('SRF IB Equity', 'SRF.NS'), ('SUNP IB Equity', 'SUNPHARMA.NS'), ('TATA IB Equity', 'TATASTEEL.NS'), ('TATACONS IB Equity', 'TATACONSUM.NS'), ('TCS IB Equity', 'TCS.NS'), ('TECHM IB Equity', 'TECHM.NS'), ('TPWR IB Equity', 'TATAPOWER.NS'), ('TRENT IB Equity', 'TRENT.NS'), ('TTAN IB Equity', 'TITAN.NS'), ('TTMT IB Equity', 'TATAMOTORS.NS'), ('UPLL IB Equity', 'UPL.NS'), ('UTCEM IB Equity', 'ULTRACEMCO.NS'), ('VEDL IB Equity', 'VEDL.NS'), ('VOLT IB Equity', 'VOLTAS.NS'), ('WPRO IB Equity', 'WIPRO.NS'), ('Z IB Equity', 'ZEEL.NS')]

display_dataframe(names_and_symbols,2007,2023,'1D')

Processed:


ERROR:yfinance:BANDHANBNK.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400
ERROR:yfinance:ADANIPOWER.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400
ERROR:yfinance:BANDHANBNK.NS: Data doesn't exist for startDate = 1199125800, endDate = 1230661800
ERROR:yfinance:BANDHANBNK.NS: Data doesn't exist for startDate = 1230748200, endDate = 1262197800
ERROR:yfinance:ADANIPOWER.NS: Data doesn't exist for startDate = 1199125800, endDate = 1230661800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1199125800, endDate = 1230661800
ERROR:yfinance:BANDHANBNK.NS: Data doesn't exist for startDate = 1262284200, endDate = 1293733800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1230748200, endDate = 1262197800
ERROR:yfinance:AUBANK.NS: Data doesn't exist for startDate = 1262284200, endDate = 1293733800
ERROR:yfinance:BANDHANBNK.NS: Data d

ADANI IB Equity
ACEM IB Equity
AUBANK IB Equity
APNT IB Equity
BHARTI IB EquityBANDHAN IB Equity

BHE IB EquityAL IB Equity

BAF IB Equity
ADSEZ IB Equity
APHS IB Equity
ARBP IB Equity


ERROR:yfinance:COALINDIA.NS: Data doesn't exist for startDate = 1167589800, endDate = 1199039400


BHFC IB Equity
AXSB IB Equity
ACC IB Equity


...

NTPC IB Equity
ONGC IB Equity


ERROR:yfinance:SBICARD.NS: Data doesn't exist for startDate = 1546281000, endDate = 1577730600


PI IB Equity


PIDI IB Equity
PAG IB Equity
PWGR IB Equity
SBICARD IB Equity
SBILIFE IB Equity
SBIN IB Equity
RIL IB Equity
SIEM IB Equity
SRCM IB Equity
SUNP IB Equity
TATA IB Equity
SRF IB Equity
TATACONS IB Equity
TECHM IB Equity
TCS IB Equity
TPWR IB Equity
TRENT IB Equity
TTMT IB Equity
TTAN IB Equity
UTCEM IB EquityUPLL IB Equity

VEDL IB Equity
VOLT IB Equity
WPRO IB Equity
Z IB Equity
A excel file named 'All_IB_Equity.xlsx'has been saved to local pc in the same directory.


ACC IB Equity                                         \
                             Open         High          Low        Close   
Date                                                                       
2007-01-01 18:30:00    792.336089   792.986747   783.010217   790.131104   
2007-01-02 18:30:00    780.769009   798.408623   771.370863   776.142212   
2007-01-03 18:30:00    783.299361   787.853883   769.202141   774.841064   
2007-01-04 18:30:00    780.769095   780.769095   755.538669   762.478821   
2007-01-07 18:30:00    762.695725   766.274215   737.429162   743.646423   
...                           ...          ...          ...          ...   
2023-12-21 18:30:00   2120.000000  2122.949951  2079.000000  2090.750000   
2023-12-25 18:30:00   2099.899902  2129.449951  2092.550049  2113.550049   
2023-12-26 18:30:00   2134.000000  2174.899902  2134.000000  2168.550049   
2023-12-27 18:30:00   2179.899902  2181.750000  2158.100098  2170.699951   
2023-12-28 18:30:00   2173.949951  2215.600098  2171.649902  2211.250000   

                                                     ACEM IB Equity  \
                       Volume Dividends Stock Splits           Open   
Date                                                                  
2007-01-01 18:30:00  261173.0       0.0          0.0     106.782407   
2007-01-02 18:30:00  429012.0       0.0          0.0     102.918594   
2007-01-03 18:30:00  448277.0       0.0          0.0     103.269846   
2007-01-04 18:30:00  319430.0       0.0          0.0     100.108516   
2007-01-07 18:30:00  311429.0       0.0          0.0      98.211706   
...                       ...       ...          ...            ...   
2023-12-21 18:30:00  444653.0       0.0          0.0     505.100006   
2023-12-25 18:30:00  255820.0       0.0          0.0     504.000000   
2023-12-26 18:30:00  875390.0       0.0          0.0     520.950012   
2023-12-27 18:30:00  339907.0       0.0          0.0     518.000000   
2023-12-28 18:30:00  428894.0       0.0          0.0     514.599976   

                                             ... WPRO IB Equity            \
                           High         Low  ...         Volume Dividends   
Date                                         ...                            
2007-01-01 18:30:00  106.782407   99.054732  ...      2135195.0       0.0   
2007-01-02 18:30:00  103.515737  101.197432  ...      4827355.0       0.0   
2007-01-03 18:30:00  103.269846   99.827523  ...      3837519.0       0.0   
2007-01-04 18:30:00  101.127162   96.631064  ...      2771786.0       0.0   
2007-01-07 18:30:00   98.211706   95.542144  ...      3708888.0       0.0   
...                         ...         ...  ...            ...       ...   
2023-12-21 18:30:00  510.399994  498.250000  ...     23040407.0       0.0   
2023-12-25 18:30:00  508.549988  502.649994  ...     27314322.0       0.0   
2023-12-26 18:30:00  523.000000  513.150024  ...     12631496.0       0.0   
2023-12-27 18:30:00  519.950012  512.049988  ...      8419978.0       0.0   
2023-12-28 18:30:00  522.450012  511.600006  ...      7980570.0       0.0   

                                 Z IB Equity                          \
                    Stock Splits        Open        High         Low   
Date                                                                   
2007-01-01 18:30:00          0.0   96.292228   96.949510   94.057461   
2007-01-02 18:30:00          0.0   94.714727   95.618491   92.710010   
2007-01-03 18:30:00          0.0   93.367302   96.505840   93.170119   
2007-01-04 18:30:00          0.0   94.320377   94.484701   92.019884   
2007-01-07 18:30:00          0.0   92.644292   93.991722   89.620779   
...                          ...         ...         ...         ...   
2023-12-21 18:30:00          0.0  266.200012  272.649994  263.750000   
2023-12-25 18:30:00          0.0  267.799988  270.250000  261.899994   
2023-12-26 18:30:00          0.0  267.399994  268.600006  261.350006   
2023-12-27 18:30: